In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import qiskit_aer.noise as noise
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_provider import IBMProvider

import numpy as np
import pandas as pd
import time

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
provider = IBMProvider()

# Get the backend (quantum device)
backend_qc = provider.get_backend('ibm_kyoto')

In [3]:
# Get the noise model from the backend
noise_model = NoiseModel.from_backend(backend_qc)

# Get coupling map from the backend
coupling_map = backend_qc.configuration().coupling_map

prob_h = 0.002  # Hypothetical error rate for H gate, slightly higher than typical single-qubit gate errors
prob_cz = 0.015  # Hypothetical error rate for CZ, estimated from CX gate error rates

error_h = noise.depolarizing_error(prob_h, 1) # Creating a 1 qubit error for the H gate
error_cz = noise.depolarizing_error(prob_cz, 2) # Creating a 2 qubit error for the CZ and CX gates

# Add these to the noise model
noise_model.add_all_qubit_quantum_error(error_h, ['h'])
noise_model.add_all_qubit_quantum_error(error_cz, ['cz', 'cx'])

# Fetching the other gates from the backend
basis_gates = noise_model.basis_gates

# Perform a noise simulation
backend = AerSimulator(noise_model=noise_model,
                       coupling_map=coupling_map,
                       basis_gates=basis_gates)

In [4]:
class QKD:
    def __init__(self, qubits, backend=backend):
        self.bits = qubits
        self.backend = backend

    def generate_random_bits(self):
        circuit = QuantumCircuit(1, 1)
        circuit.h(0)
        circuit.measure(0, 0)
        job = self.backend.run(circuit, shots=self.bits, memory=True)
        result = job.result()
        memory = result.get_memory(circuit)
        random_bits = [int(m) for m in memory]
        return random_bits
    
    def alice_sending(self):
        alice_bits = self.generate_random_bits()
        alice_bases = self.generate_random_bits()
        alice_circuit = QuantumCircuit(self.bits)
        for i in range(self.bits):
            if alice_bits[i] == 1:
                alice_circuit.x(i)  # Apply X-gate if the bit is 1
            if alice_bases[i] == 1:
                alice_circuit.h(i)  # Apply H-gate if the base is 1
        return alice_bits, alice_bases, alice_circuit
    
    def bob_receiving(self):
        alice_bits, alice_bases, alice_circuit = self.alice_sending()
        bob_bases = self.generate_random_bits()
        for i in range(self.bits):
            if bob_bases[i] == 1:
                alice_circuit.h(i)
        alice_circuit.measure_all()
        return alice_bits, alice_bases, bob_bases, alice_circuit
        
    def generate_key(self):
        alice_bits, alice_bases, bob_bases, final_circuit = self.bob_receiving()
        result = self.backend.run(final_circuit, shots=1).result()
        measurements = result.get_counts()
        bob_bits = [int(bit) for bit in list(measurements.keys())[0][::-1]]

        alice_key = []
        bob_key = []
        for i in range(self.bits):
            if alice_bases[i] == bob_bases[i]:
                alice_key.append(alice_bits[i])
                bob_key.append(bob_bits[i])
        return alice_key, bob_key

In [5]:
num_qubits = list(range(2, 26))
results = []

current_time = time.strftime("%H:%M:%S", time.localtime())
print(f"Starting: {current_time}")

for qubits in num_qubits:
    mean_length = []
    successes = 0
    for _ in range(10):
        qkd_instance = QKD(qubits=qubits)
        alice_key, bob_key = qkd_instance.generate_key()
        mean_length.append(len(alice_key))
        if alice_key == bob_key:
            successes += 1
    
    average_length = np.mean(mean_length)  # Calculate the average key length
    success_rate = successes / 10  # Calculate the success rate

    # Store the results in a list of dictionaries
    results.append({
        'Qubits': qubits,
        'Average Key Length': average_length,
        'Success Rate': success_rate
    })
    
    current_time = time.strftime("%H:%M:%S", time.localtime())
    print(f"{qubits} qubits done at: {current_time}")
    
current_time = time.strftime("%H:%M:%S", time.localtime())
print(f"Finished: {current_time}")

Starting: 21:39:30
2 qubits done at: 21:40:30
3 qubits done at: 21:41:27
4 qubits done at: 21:42:23
5 qubits done at: 21:43:20
6 qubits done at: 21:44:16
7 qubits done at: 21:45:12
8 qubits done at: 21:46:10
9 qubits done at: 21:47:08
10 qubits done at: 21:48:04
11 qubits done at: 21:49:00
12 qubits done at: 21:49:56
13 qubits done at: 21:50:53
14 qubits done at: 21:51:49
15 qubits done at: 21:52:46
16 qubits done at: 21:53:44
17 qubits done at: 21:54:41
18 qubits done at: 21:55:41
19 qubits done at: 21:56:43
20 qubits done at: 21:57:42
21 qubits done at: 21:58:47
22 qubits done at: 21:59:54
23 qubits done at: 22:01:15
24 qubits done at: 22:02:54
25 qubits done at: 22:05:16
Finished: 22:05:16


In [6]:
df = pd.DataFrame(results)

# Save the DataFrame to a txt file
df.to_csv('qkd_results_noise.txt', index=False)